#### <font color="#CC3E3E"><font size="5">**1. ▶️ SERVER HUB**

In [ ]:
# @title 1. Installation

print("Starting installation: Google Chrome...")

# 1. Update and install base dependencies
!apt-get update -qq
!apt-get install -y -qq xvfb fluxbox x11vnc libnss3 libasound2 libxss1 libgconf-2-4

# 2. Download the official Google Chrome installer. This link is stable.
print("📥 Downloading Google Chrome .deb file...")
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

# 3. VERIFY that the file has been downloaded.
print("\n🔍 Verifying the download...")
!ls -lh ./google-chrome-stable_current_amd64.deb

# 4. Install using dpkg, then fix dependencies.
print("\n🛠️ Step 1/2: Attempting installation with dpkg...")
!dpkg -i ./google-chrome-stable_current_amd64.deb

print("🛠️ Step 2/2: Fixing broken dependencies...")
!apt-get -f install -y -qq

print("✅ Google Chrome installed successfully.")

# 5. Install Gradio & other tools
!pip install -q gradio
!wget -q -O noVNC.zip https://github.com/novnc/noVNC/archive/refs/tags/v1.4.0.zip
!unzip -q noVNC.zip -d /content/
!mv /content/noVNC-1.4.0 /content/noVNC
!ln -s /content/noVNC/utils/novnc_proxy /usr/local/bin/novnc_proxy
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared

print("✅ All installations complete.")

In [ ]:
# @title 2. Run Google Chrome Server & Gradio

import os
import subprocess
import threading
import time
import re
import gradio as gr

# --- Configuration ---
VNC_PORT = 5900
NOVNC_PORT = 6080
GRADIO_PORT = 7860
VNC_PASSWORD = "password"

# --- Global Variables ---
novnc_public_url = None
gradio_public_url = None

# --- Helper Functions ---
def kill_process_on_port(port):
    print(f"Attempting to clear port {port}...")
    try:
        subprocess.run(f"fuser -k {port}/tcp", shell=True, check=False, capture_output=True)
        print(f"✅ Port {port} cleared successfully.")
        time.sleep(1)
    except FileNotFoundError:
        print(f"ℹ️ 'fuser' command not found. Skipping port clearing.")

def start_servers():
    """Starts all servers in the background with the correct order and delays."""
    os.environ['DISPLAY'] = ':1'
    print("🚀 1. Starting Virtual Display (Xvfb)...")
    subprocess.Popen(['Xvfb', ':1', '-screen', '0', '1280x800x24', '-ac'])
    time.sleep(3)
    print("🚀 2. Starting Window Manager (Fluxbox)...")
    subprocess.Popen(['fluxbox'])
    time.sleep(2)
    print("🚀 3. Starting VNC Server (x11vnc)...")
    subprocess.Popen(['x11vnc', '-display', ':1', '-rfbport', str(VNC_PORT), '-passwd', VNC_PASSWORD, '-ncache', '10', '-forever'])
    time.sleep(2)
    print("🚀 4. Starting noVNC Proxy...")
    subprocess.Popen(['/content/noVNC/utils/novnc_proxy', '--listen', str(NOVNC_PORT), '--vnc', f'localhost:{VNC_PORT}'])
    time.sleep(2)

    # --- MAIN CHANGE: Running google-chrome ---
    print("🚀 5. Starting Google Chrome...")
    subprocess.Popen(['google-chrome', '--no-sandbox', '--disable-dev-shm-usage', '--disable-gpu', '--start-maximized'])

    print("✅ All server start commands have been issued.")

def start_cloudflared_tunnel(port, target_url_var_name):
    global novnc_public_url, gradio_public_url
    p = subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    for line in p.stdout:
        if ".trycloudflare.com" in line:
            public_url = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line).group(0)
            if target_url_var_name == 'novnc': novnc_public_url = public_url
            elif target_url_var_name == 'gradio': gradio_public_url = public_url
            break

# --- Run Everything ---
kill_process_on_port(GRADIO_PORT)
kill_process_on_port(NOVNC_PORT)

start_servers()
time.sleep(5)

novnc_tunnel_thread = threading.Thread(target=start_cloudflared_tunnel, args=(NOVNC_PORT, 'novnc'))
novnc_tunnel_thread.daemon = True
novnc_tunnel_thread.start()
gradio_tunnel_thread = threading.Thread(target=start_cloudflared_tunnel, args=(GRADIO_PORT, 'gradio'))
gradio_tunnel_thread.daemon = True
gradio_tunnel_thread.start()

print("⏳ Waiting for public URLs from Cloudflare...")
while not novnc_public_url or not gradio_public_url: time.sleep(1)

print("\n\n=======================================================================")
print("✅ EVERYTHING IS READY!")
print("Open the Gradio link below in your browser — make sure to open it in an incognito/private tab.")
print(f"  ➡️  {gradio_public_url}  ⬅️")
print("=======================================================================\n\n")

with gr.Blocks(title="Browser in Colab") as demo:
    gr.Markdown(f"""
    # 🌐 Google Chrome in Colab
    This is a Google Chrome browser session running entirely on a Google Colab server.
    **VNC Password if requested:** `password`
    """)
    gr.HTML(f"""<iframe src="{novnc_public_url}/vnc.html" style="width: 100%; height: 800px; border: 1px solid #ccc;"></iframe>""")

demo.launch(server_port=GRADIO_PORT, share=False)

In [3]:
# @title 3. Link Downloads Folder
# @markdown This command creates a shortcut to the real Downloads folder so you can see it in the file panel.
!ln -s /root/Downloads /content/Downloads

#### <font color="#CC3E3E"><font size="5">**2. ▶️ HUGGINGFACE HUB**

In [ ]:
# @title ## **1. Setup Environment**
# @markdown Run this cell to install the required libraries.
from IPython.display import clear_output, display, HTML
import os

print("⏳ Installing required libraries...")

# Combine installation commands for efficiency
!pip install -q huggingface_hub
!apt-get -y install -qq aria2

clear_output()

# Option to mount Google Drive
mount_google_drive = False  # @param {type: "boolean"}
if mount_google_drive:
    from google.colab import drive
    print("🚀 Mounting Google Drive...")
    drive.mount('/content/drive')
    print("✅ Google Drive successfully mounted at /content/drive")
else:
    print("☑️ Google Drive not mounted. You can upload files from your computer in a later step.")

print("\n\n✅ Environment is ready! Please proceed to the next cell.")

In [ ]:
# @title ## **2. Configure & Login to Hugging Face**
from huggingface_hub import login, HfApi, HfFolder
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
import getpass

# --- Configuration Form ---
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
huggingface_token = ""  # @param {type:"string"}
# @markdown ---
# @markdown **Repository Details**
# @markdown Just provide the repo name, not the full `username/repo-name`.
model_repo_name = ""  # @param{type:"string"}
# @markdown Leave this empty if you don't want to create a dataset repo.
dataset_repo_name = ""  # @param{type:"string"}
# @markdown ---
# @markdown **Optional Information**
organization_name = ""  # @param{type:"string"}
make_private = False  # @param{type:"boolean"}


# --- Functions and Logic ---
def setup_hf_hub(token, model_repo, dataset_repo, org_name, is_private):
    """Logs into the Hub and creates repositories if they don't exist."""
    if not token or not token.startswith("hf_"):
        print("❌ Invalid Hugging Face token. Please provide a valid token.")
        return None, None, None

    print("🔑 Logging into Hugging Face Hub...")
    login(token=token, add_to_git_credential=True)
    api = HfApi()
    user = api.whoami()
    username = user["name"]
    print(f"👋 Successfully logged in as: {username}")

    model_repo_id = None
    dataset_repo_id = None

    # Determine the repo owner (user or organization)
    repo_owner = org_name if org_name else username

    # Create model repository if a name is provided
    if model_repo:
        model_repo_id = f"{repo_owner}/{model_repo.strip()}"
        print(f"\n🔍 Checking model repository: {model_repo_id}")
        try:
            validate_repo_id(model_repo_id)
            api.create_repo(repo_id=model_repo_id, repo_type="model", private=is_private, exist_ok=True)
            print(f"✅ Model repository is ready: https://huggingface.co/{model_repo_id}")
        except HfHubHTTPError as e:
            print(f"❌ Failed to create or access model repository: {e}")
            model_repo_id = None

    # Create dataset repository if a name is provided
    if dataset_repo:
        dataset_repo_id = f"{repo_owner}/{dataset_repo.strip()}"
        print(f"\n🔍 Checking dataset repository: {dataset_repo_id}")
        try:
            validate_repo_id(dataset_repo_id)
            api.create_repo(repo_id=dataset_repo_id, repo_type="dataset", private=is_private, exist_ok=True)
            print(f"✅ Dataset repository is ready: https://huggingface.co/datasets/{dataset_repo_id}")
        except HfHubHTTPError as e:
            print(f"❌ Failed to create or access dataset repository: {e}")
            dataset_repo_id = None

    return api, model_repo_id, dataset_repo_id

# --- Execution ---
if not model_repo_name and not dataset_repo_name:
    print("❌ You must provide either a `model_repo_name` or a `dataset_repo_name`.")
else:
    api, model_repo_id, dataset_repo_id = setup_hf_hub(
        huggingface_token,
        model_repo_name,
        dataset_repo_name,
        organization_name,
        make_private
    )

In [ ]:
# @title ## **3. Upload to Hugging Face**
from pathlib import Path

# --- Upload Form ---
# @markdown **Path to the file or folder to upload**
path_to_upload = ""  # @param {type:"string"}

# @markdown **Path in repository (leave blank to use original name)**
path_in_repo = ""  # @param {type:"string"}

# @markdown **Commit message for this upload**
commit_message = "feat: Upload model from Colab"  # @param {type:"string"}


# --- Functions and Logic ---
def upload_to_hf_hub(api_client, repo_id, local_path_str, repo_path_str, commit_msg):
    """Uploads a file or folder to a repository."""
    local_path = Path(local_path_str)

    if not local_path.exists():
        print(f"❌ Error: Path '{local_path_str}' was not found. Please double-check the path.")
        return

    # Determine the destination path in the repo
    if not repo_path_str:
        # Use the original name if the repo path is not specified
        repo_path_str = local_path.name

    is_folder = local_path.is_dir()
    upload_type = "folder" if is_folder else "file"

    print(f"\n🚀 Starting {upload_type} upload: '{local_path.name}'")
    print(f"Repository: https://huggingface.co/{repo_id}")
    print(f"Path in Repo: {repo_path_str}")
    print("Please wait...")

    try:
        if is_folder:
            api_client.upload_folder(
                folder_path=str(local_path),
                path_in_repo=repo_path_str,
                repo_id=repo_id,
                commit_message=commit_msg,
                ignore_patterns=[".ipynb_checkpoints"],
            )
            final_url = f"https://huggingface.co/{repo_id}/tree/main/{repo_path_str}"
        else:
            api_client.upload_file(
                path_or_fileobj=str(local_path),
                path_in_repo=repo_path_str,
                repo_id=repo_id,
                commit_message=commit_msg,
            )
            final_url = f"https://huggingface.co/{repo_id}/blob/main/{repo_path_str}"

        print(f"\n🎉 Upload successful!")
        print(f"Check it out here: {final_url}")

    except Exception as e:
        print(f"❌ An error occurred during the upload: {e}")


# --- Execution ---
if 'api' not in locals() or 'model_repo_id' not in locals():
    print("❌ Please run Cell 2 (Configure & Login) first.")
elif not model_repo_id:
    print("❌ Model repository is not defined. Please ensure `model_repo_name` in Cell 2 is correct.")
elif not path_to_upload:
    print("❌ Please fill in the `path_to_upload` with the path to your file or folder.")
else:
    upload_to_hf_hub(
        api_client=api,
        repo_id=model_repo_id,
        local_path_str=path_to_upload,
        repo_path_str=path_in_repo,
        commit_msg=commit_message
    )